In [1]:
import traceback
from flask import render_template, request, redirect, url_for
import logging.config
#from app.models import UserInput
from flask import Flask
from src.add_movies import UserInput
from flask_sqlalchemy import SQLAlchemy
from src.helpers import helpers
import sqlalchemy as sql
import pandas as pd

# Initialize the Flask application
app = Flask(__name__)

In [2]:
# Configure flask app from flask_config.py
app.config.from_pyfile('./config/flask_config.py')

# Define LOGGING_CONFIG in flask_config.py - path to config file for setting
# up the logger (e.g. config/logging/local.conf)
logging.config.fileConfig(app.config["LOGGING_CONFIG"])
logger = logging.getLogger("movieRecapp")
logger.debug('Test log')

# Initialize the database
#db = SQLAlchemy(app)

In [3]:
logging.config.fileConfig(app.config["LOGGING_CONFIG"])
logger = logging.getLogger("movieRecapp")
logger.debug('Test log')

# Initialize the database
db = SQLAlchemy(app)


In [ ]:
sq

In [4]:
userinput = UserInput(movie="Rush Hour",rating=5)

In [7]:
import os

In [8]:
os.environ['MYSQL_USER']='root'
os.environ['MYSQL_PASSWORD']='movierec'
os.environ['MYSQL_HOST']="mysql-nw-schockalingam.c6vsb8kehwgn.us-east-2.rds.amazonaws.com"
os.environ['MYSQL_PORT']="3306"


In [9]:

DATABASE_NAME = 'msia423'
conn_type = "mysql+pymysql"
user = os.environ.get("MYSQL_USER")
password = os.environ.get("MYSQL_PASSWORD")
host = os.environ.get("MYSQL_HOST")
port = os.environ.get("MYSQL_PORT")
engine_string = "{}://{}:{}@{}:{}/{}".format(conn_type, user, password, host, port, DATABASE_NAME)


In [122]:
query = "SELECT * FROM userinput"
df = pd.read_sql(query,con=engine)

In [123]:
df

,id,movie,rating
0,14,Avengers,5
1,15,Rush Hour,5


In [100]:
session = helpers.get_session(engine_string=engine_string)

In [8]:
import movierecommender as mr

In [9]:
newmr = mr.MovieRecommender(model_config = './config/test_model_config.yml')

C:\Users\Sabarish\Google Drive\HomeWork\Data Science\MSiA 423\Project Git\movierecommender.py:26: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)
2019-06-09 21:55:53,153 movierec-score INFO     Configuration file loaded from ./config/test_model_config.yml


In [109]:
df2 = df.merge(newmr.movies,left_on='movie',right_on='title',how='left')


In [124]:
df2 = df2.dropna()
df2.loc[:,['movieId']] = df2.loc[:,['movieId']].astype(int)
df2.loc[:,['rating']] = df2.loc[:,['rating']].astype(float)

In [112]:
newuserid = max(newmr.ratings.userId)+1

In [85]:
newuserid

611

In [113]:
df2['userId'] = newuserid

In [115]:
df2[['userId','movieId','rating']]

,userId,movieId,rating
1,611,2273,5.0


In [117]:
newratings = pd.concat([newmr.ratings[['userId','movieId','rating']],df2[['userId','movieId','rating']]])

In [119]:
# reader should know what scale is
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(newratings[['userId','movieId','rating']],reader)
algo = SVD()

NameError: name 'Reader' is not defined

In [ ]:
trainset = data.build_full_trainset()
algo.fit(trainset)

In [4]:
import src.process_data as proda

In [5]:
import movierecommender as mr

In [6]:
newmr = mr.MovieRecommender()

C:\Users\Sabarish\Google Drive\HomeWork\Data Science\MSiA 423\Project Git\movierecommender.py:26: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)
2019-06-09 22:07:20,417 movierec-score INFO     Configuration file loaded from config/test_model_config.yml


In [10]:
proda.procuserinput(newmr.movies,newmr.ratings,engine_string=engine_string)

,userId,movieId,rating
1,611,2273,5.0


In [7]:
import src.process_data as proda

In [6]:
proda.getuserinput(engine_string)

,id,movie,rating
0,14,Avengers,5
1,15,Rush Hour,5
